Notebook para el proceso etl del proyecto

In [2]:
import pandas as pd
import numpy as np
import re, ast



ETL para el dataset movies_dataset.csv

In [ ]:
df_movies = pd.read_csv('C:\Soy-Henry\Film recommendation model\Data\movies_dataset.csv')
df_movies.head(3)

Detalle de pasos para el ETL del dataset movies_dataset.csv
1) Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.
2) Los valores nulos del campo release date deben eliminarse.
3) Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.
4) De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.
5) Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.
6) Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.


In [4]:
# paso 1
df_movies  = df_movies.drop(columns= ["video","imdb_id","adult","original_title","poster_path","homepage"])

In [ ]:
df_movies.info()

In [6]:
#paso 2
df_movies.dropna(subset=['release_date'], inplace=True)
# eliminamos 3 registros de title nulos
df_movies.dropna(subset=['title'], inplace=True)

In [ ]:
df_movies.info()

In [8]:
#paso 3
df_movies.fillna({'revenue': 0}, inplace=True)

In [ ]:
df_movies.fillna({'budget': 0}, inplace=True)

In [110]:
#paso 4
#df_movies['release_date'] = pd.to_datetime(df_movies['release_date']).dt.strftime('%Y%m%d')
print(df_movies['release_date'][:5])

0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
Name: release_date, dtype: object


In [ ]:
#paso 4 b
df_movies['release_year']=df_movies['release_date'].str[:4]
df_movies["release_date"] = pd.to_datetime(df_movies["release_date"], infer_datetime_format=False)
df_movies

In [10]:
# paso 5a - determinamos los tipos de variables
a=df_movies['revenue'].loc[1]
b=df_movies['budget'].loc[1]

print(type(a)) #float
print(type(b)) #str

<class 'numpy.float64'>
<class 'str'>


In [11]:
# Paso 5b
# Convertimos df_movies['budget'] tipo string en float64 
df_movies['budget'] = pd.to_numeric(df_movies['budget'],errors = 'coerce')


In [6]:
# paso 5c
tot_sel=df_movies['id'].count() # obtengo el total de filas del dataframe

for ind_pel in range(0,tot_sel):
    budget = df_movies['budget'].iloc[ind_pel]
    revenue = df_movies['revenue'].iloc[ind_pel]
    if budget == 0 or revenue == 0:
        df_movies['roi'].iloc[ind_pel] = 0
    else:
        df_movies['roi'].iloc[ind_pel] = ((revenue-budget)/budget)*100 # si budget o revenue no es cero determino la ganancia y luego la pongo en porcentaje  

In [218]:
# paso 6
# determinamos que el campo belongs_to_collection no es necesario para este proyecto al igual que los campos genres,homepage,original_language,overview,poster_path,
# production_companies,production_countries,spoken_languages,status,video,tagline del dataset movies_dataset

df_movies.drop(columns= ["belongs_to_collection","genres","original_language","overview","production_companies","production_countries","spoken_languages","status","tagline","runtime"],inplace=True)


In [ ]:
df_movies = pd.read_csv('C:\Soy-Henry\Film recommendation model\Data\movies_final.csv')
df_movies.isnull().sum()

In [45]:
### finalizado el proceso etl del dataset movies_dataset volcamos el resultado obtenido a un nuevo archivo csv llamado movies_final
df_movies.to_csv('C:\Soy-Henry\Film recommendation model\Data\movies_final.csv',index=False)

 ETL PARA EL DATASET CREDITS.CSV

In [2]:
df_credits = pd.read_csv('C:\Soy-Henry\Film recommendation model\Data\credits.csv')
df_credits.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [230]:
cast=df_credits['cast'].iloc[0]
cast


"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [232]:
cadena = cast.strip('\'\'')

# Convertir la cadena en un diccionario utilizando ast.literal_eval()
diccionario = ast.literal_eval(cadena)
diccionario

[{'cast_id': 14,
  'character': 'Woody (voice)',
  'credit_id': '52fe4284c3a36847f8024f95',
  'gender': 2,
  'id': 31,
  'name': 'Tom Hanks',
  'order': 0,
  'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'},
 {'cast_id': 15,
  'character': 'Buzz Lightyear (voice)',
  'credit_id': '52fe4284c3a36847f8024f99',
  'gender': 2,
  'id': 12898,
  'name': 'Tim Allen',
  'order': 1,
  'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'},
 {'cast_id': 16,
  'character': 'Mr. Potato Head (voice)',
  'credit_id': '52fe4284c3a36847f8024f9d',
  'gender': 2,
  'id': 7167,
  'name': 'Don Rickles',
  'order': 2,
  'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'},
 {'cast_id': 17,
  'character': 'Slinky Dog (voice)',
  'credit_id': '52fe4284c3a36847f8024fa1',
  'gender': 2,
  'id': 12899,
  'name': 'Jim Varney',
  'order': 3,
  'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'},
 {'cast_id': 18,
  'character': 'Rex (voice)',
  'credit_id': '52fe4284c3a36847f8024fa5',
  'gender': 2,
  'id': 12900,
 

In [272]:
#### iteramos el df_credits y creamos el dataframe df_casting
df_casting = pd.DataFrame(columns=['id_actor', 'name_actor', 'id_film'])
len(df_credits)
for i in range(0,len(df_credits)):
    idfilm=df_credits['id'].iloc[i]
    cast=df_credits['cast'].iloc[i]
    cast = cast.strip('\'\'')
    diccionario = ast.literal_eval(cast)
    df = pd.DataFrame(diccionario, columns=['id', 'name'])
    df['id_film']=idfilm
    df=df.rename(columns={'id':'id_actor','name':'name_actor'})
    clave=['id_actor','name_actor','id_film']
    df_casting=pd.merge(df_casting, df, on=clave,how="outer")

    
    

In [274]:
# volcamos el dataframe df_casting en el archivo casting_final.csv
df_casting.to_csv('C:\Soy-Henry\Film recommendation model\Data\casting_final.csv',index=False)

In [3]:
crew=df_credits['crew'].iloc[0]
crew

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

In [5]:
crew = crew.strip('\'\'')

# Convertir la cadena en un diccionario utilizando ast.literal_eval()
diccionario = ast.literal_eval(crew)
diccionario

[{'credit_id': '52fe4284c3a36847f8024f49',
  'department': 'Directing',
  'gender': 2,
  'id': 7879,
  'job': 'Director',
  'name': 'John Lasseter',
  'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f4f',
  'department': 'Writing',
  'gender': 2,
  'id': 12891,
  'job': 'Screenplay',
  'name': 'Joss Whedon',
  'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f55',
  'department': 'Writing',
  'gender': 2,
  'id': 7,
  'job': 'Screenplay',
  'name': 'Andrew Stanton',
  'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f5b',
  'department': 'Writing',
  'gender': 2,
  'id': 12892,
  'job': 'Screenplay',
  'name': 'Joel Cohen',
  'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f61',
  'department': 'Writing',
  'gender': 0,
  'id': 12893,
  'job': 'Screenplay',
  'name': 'Alec Sokolow',
  'profile_path': '/v79vlRYi94BZUQnkkyzn

In [9]:
df = pd.DataFrame(diccionario)  # Crear un DataFrame a partir del diccionario

filtered_df = df[df['job'] == 'Director']  # Filtrar por 'job' igual a 'Director'

directors = filtered_df['name'].tolist()  # Extraer los nombres de los directores
filtered_df
#directors

,credit_id,department,gender,id,job,name,profile_path
0,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg


In [15]:
#### iteramos el df_credits y creamos el dataframe df_dir donde guardaremos el director de cada pelicula
df_dir = pd.DataFrame(columns=['id_director', 'name_director', 'id_film'])

for i in range(0,len(df_credits)):
    id_film=df_credits['id'].iloc[i] # tomo el id de la película
    director=df_credits['crew'].iloc[i] # tomo el registro crew donde de se encuentran los datos del director de la película
    director = director.strip('\'\'')
    diccionario = ast.literal_eval(director) # finalizo el formateo para dejarlo como diccionario
    for person in diccionario:
        if person['department'] == 'Directing':
            id_director = person["id"]
            name_director = person["name"]
            df_dir = df_dir.append(pd.Series([id_director, name_director, id_film],index=['id_director', 'name_director', 'id_film']),ignore_index=True)
            break # una vez que encuentro al director salgo del loop

In [16]:
# volcamos el dataframe df_casting en el archivo directors_final.csv
df_dir.to_csv('C:\Soy-Henry\Film recommendation model\Data\directors_final.csv',index=False)
